In [1]:
!pip install transformers datasets accelerate -U peft evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.2 MB/s eta 0:00:00


In [3]:
from transformers import BertTokenizerFast, BertForQuestionAnswering, TrainingArguments, Trainer
from datasets import load_dataset
from peft import PeftModelForQuestionAnswering, PromptEncoderConfig, PromptEncoder
from collections import Counter
import numpy as np
import re
import string

dataset = load_dataset("legacy107/cpgQA")
train_dataset = dataset['train']
eval_dataset = dataset['test']


# Load the fast tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    encodings = tokenizer(
        examples['question'],
        examples['context'],
        truncation=True,
        padding='max_length',
        max_length=492,
        return_offsets_mapping=True
    )

    start_positions = []
    end_positions = []

    for i, (context, answer, answer_start) in enumerate(zip(examples['context'], examples['answer'], examples['answer_start'])):
        start_position = None
        end_position = None

        start_idx = answer_start
        end_idx = start_idx + len(answer)

        offset_mapping = encodings['offset_mapping'][i]

        for j, (offset_start, offset_end) in enumerate(offset_mapping):
            if offset_start <= start_idx and offset_end > start_idx:
                start_position = j
            if offset_start < end_idx and offset_end >= end_idx:
                end_position = j
                break

        if start_position is not None and end_position is not None:
            start_positions.append(start_position)
            end_positions.append(end_position)
        else:
            start_positions.append(0)
            end_positions.append(0)

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return encodings

# Tokenize the dataset first
train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# P-tuning Configuration
prompt_encoder_config = PromptEncoderConfig(
    peft_type="P_TUNING",
    task_type="QUESTION_ANS",
    num_virtual_tokens=20,
    token_dim=768,
    num_transformer_submodules=1,
    num_attention_heads=12,
    num_layers=12,
    encoder_reparameterization_type="MLP",
    encoder_hidden_size=768,
)

prompt_encoder = PromptEncoder(prompt_encoder_config)

# Create the model
model = BertForQuestionAnswering.from_pretrained("csarron/bert-base-uncased-squad-v1")
peft_model = PeftModelForQuestionAnswering(model, prompt_encoder_config)
peft_model.print_trainable_parameters()

# Training arguments
training_args = TrainingArguments(
    evaluation_strategy="epoch",
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    learning_rate=5e-5,
    save_strategy="epoch",
    load_best_model_at_end=True,
    greater_is_better=True,
    fp16=True,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Fine-tune
trainer.train()

# Compute predictions
predictions = trainer.predict(eval_dataset)

# Get the predicted start and end positions
pred_start = predictions.predictions[0].argmax(axis=-1)
pred_end = predictions.predictions[1].argmax(axis=-1)

# Extract true start and end positions from the evaluation dataset
true_start_positions = [item['start_positions'] for item in eval_dataset]
true_end_positions = [item['end_positions'] for item in eval_dataset]


# Calculate F1
f1s = []
for i in range(0, len(pred_start)):
    pred_span = set(range(pred_start[i], pred_end[i] + 1))
    true_span = set(range(true_start_positions[i], true_end_positions[i] + 1))

    num_same = len(pred_span.intersection(true_span))
    if num_same == 0:
        f1s.append(0)
        continue

    precision = float(num_same) / float(len(pred_span))
    recall = float(num_same) / float(len(true_span))
    f1 = (2 * precision * recall) / (precision + recall)
    f1s.append(f1)

print('Average F1 Score: {:.3f}'.format(np.mean(f1s)))

# Calculate Exact Match
total_correct = 0
for i in range(0, len(pred_start)):
    matches = 0
    if pred_start[i] == true_start_positions[i]:
        matches += 1
    if pred_end[i] == true_end_positions[i]:
        matches += 1
    total_correct += matches

total_indeces = 2 * len(pred_start)  # because both start and end are considered
print('Correctly predicted indeces: {:,} of {:,} ({:.2%})'.format(
    total_correct,
    total_indeces,
    float(total_correct) / float(total_indeces)
))




Map:   0%|          | 0/110 [00:00<?, ? examples/s]

Some weights of the model checkpoint at csarron/bert-base-uncased-squad-v1 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


trainable params: 1,790,212 || all params: 110,681,860 || trainable%: 1.6174393888935368


Epoch,Training Loss,Validation Loss
1,No log,2.516230
2,No log,2.441141
3,No log,2.366130


Average F1 Score: 0.645
Correctly predicted indeces: 124 of 220 (56.36%)


In [ ]:
#evaluating models against SQUAD benchmark
import pandas as pd
from datasets import load_dataset
from evaluate import evaluator
from transformers import pipeline, BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

models = [
    "./results/checkpoint-126",
    "./results/checkpoint-42",
    "./results/checkpoint-84"
]

data = load_dataset("squad", split="validation")
task_evaluator = evaluator("question-answering")

results = []
for model in models:
    results.append(
        task_evaluator.compute(
            model_or_pipeline=model,
            tokenizer=tokenizer,
            data=data,
            metric="squad",
            squad_v2_format=False
            )
        )

df = pd.DataFrame(results, index=models)
df[["exact_match", "f1", "total_time_in_seconds", "samples_per_second", "latency_in_seconds"]]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Some weights of the model checkpoint at ./results/checkpoint-126 were not used when initializing BertForQuestionAnswering: ['qa_outputs.modules_to_save.default.weight', 'qa_outputs.original_module.bias', 'qa_outputs.original_module.weight', 'qa_outputs.modules_to_save.default.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at ./results/checkpoint-126 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-st

Some weights of the model checkpoint at ./results/checkpoint-42 were not used when initializing BertForQuestionAnswering: ['qa_outputs.modules_to_save.default.weight', 'qa_outputs.original_module.bias', 'qa_outputs.original_module.weight', 'qa_outputs.modules_to_save.default.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at ./results/checkpoint-42 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stre

,exact_match,f1,total_time_in_seconds,samples_per_second,latency_in_seconds
./results/checkpoint-126,5.212867,23.267651,165.757347,63.767913,0.015682
./results/checkpoint-42,1.106906,17.985008,166.000259,63.674600,0.015705
./results/checkpoint-84,0.993377,25.288934,166.526448,63.473401,0.015755
